[View in Colaboratory](https://colab.research.google.com/github/restrepo/orcid/blob/master/Orcid.ipynb)

# Búsqueda de autores en Orcid
See [search-orcid-registry](http://members.orcid.org/api/tutorial/search-orcid-registry)

ORCID: https://github.com/ORCID/python-orcid

## Instalar paquetes y cargar datos

In [0]:
!pip install wosplus openpyxl xlrd orcid > /dev/null

In [20]:
%%writefile drive.cfg
[FILES]
Base_de_datos_investigadores_Definitiva.csv=12oalgUeKhpvzkTPBP8pXCeHTrF-KO223dy9ov9w9QKs 

Overwriting drive.cfg


In [0]:
import requests
import pandas as pd
import wosplus as wp
pd.set_option('display.max_colwidth',500)

In [0]:
bdi=wp.read_drive_excel('Base_de_datos_investigadores_Definitiva.csv')

In [23]:
bdi[:2]

,CÉDULA,NOMBRE COMPLETO,NOMBRES,PRIMER APELLIDO,SEGUNDO APELLIDO,FACULTAD,DEPARTAMENTO,GRUPO
0,71621971,Bernardo Maria Bustamante Cardona,Bernardo Maria,Bustamante,Cardona,Facultad de Artes,Departamento de Artes Visuales,Arteducación
1,71652139,Carlos Mario Jaramillo Ramirez,Carlos Mario,Jaramillo,Ramirez,Facultad de Artes,Departamento de Artes Visuales,DIDARTE. Didáctica de las artes


## Obtener el token del API

In [0]:
# From https://github.com/ORCID/python-orcid
import orcid
api = orcid.PublicAPI('APP-R8J5P57TTE983P54', 'c6e1dd48-80d3-4dd9-9336-fc41728ac8fe', sandbox=False)
#token = api.get_token_from_authorization_code(authorization_code,
#                                              redirect_uri)


In [0]:
search_token = api.get_search_token_from_orcid()

In [26]:
search_token

'22121193-5ce4-4c4e-938d-629d5dcf6af9'

https://pub.orcid.org

### Comprobar que funciona

In [0]:
HEADERS_orcid={'Content-type': 'application/vnd.orcid+json',
    'Access token': search_token}

In [0]:
r=requests.get('https://pub.orcid.org/v2.1/search/?q=newman', headers=HEADERS_orcid)

Muestre los dos primeros elementos de la lista de resultados

In [29]:
pd.read_json(r.text)[:2]

,num-found,result
0,948,{'orcid-identifier': {'uri': 'https://orcid.or...
1,948,{'orcid-identifier': {'uri': 'https://orcid.or...


## Definir la función de búsqueda. 
Ver ayuda en la documentación del API

In [0]:
import pandas as pd
import time
import random
def search_orcid(q='family-name:Taylor+AND+given-names:Michael',search_token=''):
    '''
    Search API orcid in json for query q: (see examples here: http://members.orcid.org/api/tutorial/search-orcid-registry)
    Example: 
    q=family-name:Taylor+AND+given-names:Michael
    '''
    HEADERS_orcid={'Content-type': 'application/vnd.orcid+json',
    'Access token': search_token}
    r=requests.get('https://pub.orcid.org/v2.1/search/?q={}'.format(q),
                   headers=HEADERS_orcid)
    
    # wait for next request
    time.sleep(random.randint(1,3))
    if r.status_code==200:
        return pd.read_json(r.text)
    else:
        return pd.DataFrame()

## Formulas de búsqueda

### Búsqueda por nombre completo y afiliación

Muestra de búsqueda por nombre para investigador de la UdeA

In [0]:
df=search_orcid('family-name:Restrepo+AND+given-names:Diego+AND+affiliation-org-name:Universidad+de+Antioquia',
                search_token)#+AND+given-names:Diego')#+AND+affiliation-org-name:Universidad+de+Antioquia')

Muestre los primeros dos resultados de la búsqueda

In [32]:
df[:2]

,num-found,result
0,6,{'orcid-identifier': {'uri': 'https://orcid.or...
1,6,{'orcid-identifier': {'uri': 'https://orcid.or...


#### Prepare búsqueda investigadores UdeA

In [0]:
apellido='Restrepo'
nombre='Diego'
affiliation='Universidad de Antioquia'

In [0]:
df=search_orcid('family-name:{}+AND+given-names:{}+AND+affiliation-org-name:{}'.format(
                  apellido.replace(' ','+'), nombre.replace(' ','+'), affiliation.replace(' ','+') ) ,
                search_token)

In [36]:
df[:2]

,num-found,result
0,6,{'orcid-identifier': {'uri': 'https://orcid.or...
1,6,{'orcid-identifier': {'uri': 'https://orcid.or...


### Ensayo con una entrada de la base de datos de profesores

Además de afiliación:
1. Nombres y apellidos completo
2. Primer nombre y primer apellido
3. Segundo nombre y primer appellido

Se va a hacer un loop sobre la base de datos de investigadores de la UdeA. Se ilustrará primero lo que pasa con la primera entrada  de la lista cuando se búsca de la forma 1.

In [37]:
bdi[:1]

,CÉDULA,NOMBRE COMPLETO,NOMBRES,PRIMER APELLIDO,SEGUNDO APELLIDO,FACULTAD,DEPARTAMENTO,GRUPO
0,71621971,Bernardo Maria Bustamante Cardona,Bernardo Maria,Bustamante,Cardona,Facultad de Artes,Departamento de Artes Visuales,Arteducación


In [0]:
i=0

In [39]:
bdi.loc[i,'NOMBRES'].strip()

'Bernardo Maria'

In [40]:
bdi.loc[i,'PRIMER APELLIDO']+' '+bdi.loc[i,'SEGUNDO APELLIDO']

'Bustamante Cardona'

In [0]:
apellido=bdi.loc[i,'PRIMER APELLIDO'].strip()
apellido_completo=apellido+' '+bdi.loc[i,'SEGUNDO APELLIDO'].strip()
nombre_completo=bdi.loc[i,'NOMBRES'].strip()
affiliation='Universidad de Antioquia'
df=search_orcid('family-name:{}+AND+given-names:{}+AND+affiliation-org-name:{}'.format(
                  apellido_completo.replace(' ','+'), 
                  nombre_completo.replace(' ','+'), affiliation.replace(' ','+') ) ,
                search_token)

In [42]:
df

,num-found,result


Ahora intentamos con la búsqueda 2. 

Para ello añadimos una columna con el primer nombre al DataFrame

In [0]:
bdi['Primer_NOMBRE']=bdi.NOMBRES.str.split().str[0]
nombre=bdi.loc[i,'Primer_NOMBRE'].strip()

In [44]:
bdi[:1]

,CÉDULA,NOMBRE COMPLETO,NOMBRES,PRIMER APELLIDO,SEGUNDO APELLIDO,FACULTAD,DEPARTAMENTO,GRUPO,Primer_NOMBRE
0,71621971,Bernardo Maria Bustamante Cardona,Bernardo Maria,Bustamante,Cardona,Facultad de Artes,Departamento de Artes Visuales,Arteducación,Bernardo


In [0]:
df=search_orcid('family-name:{}+AND+given-names:{}+AND+affiliation-org-name:{}'.format(
                  apellido.replace(' ','+'), 
                  nombre.replace(' ','+'), affiliation.replace(' ','+') ) ,
                search_token)

In [46]:
df

,num-found,result


Finalmente intentamos con la búsqueda 3.

In [47]:
bdi['Segundo_NOMBRE']=bdi.NOMBRES.str.split().str[1].fillna('')
segundo_nombre=bdi.loc[i,'Segundo_NOMBRE'].strip()
segundo_nombre

'Maria'

In [0]:
df=search_orcid('family-name:{}+AND+given-names:{}+AND+affiliation-org-name:{}'.format(
                  apellido.replace(' ','+'), 
                  segundo_nombre.replace(' ','+'), affiliation.replace(' ','+') ) ,
                search_token)

In [49]:
df

,num-found,result
0,2,{'orcid-identifier': {'uri': 'https://orcid.or...
1,2,{'orcid-identifier': {'uri': 'https://orcid.or...


## Búsqueda definitiva sobre todos los autores
TODO: Hacer que el programa corra desde aquí

In [50]:
dft=pd.DataFrame()
affiliation='Universidad de Antioquia'
# Adicionar dos columnas nuevas al DataFrame para las búsquedas 2 y 3:
bdi['Primer_NOMBRE']=bdi.NOMBRES.str.split().str[0]
bdi['Segundo_NOMBRE']=bdi.NOMBRES.str.split().str[1].fillna('')
for i in bdi.index:
    df=pd.DataFrame()
    print(i)
    nombre_completo=bdi.loc[i,'NOMBRES'].strip()
    nombre=bdi.loc[i,'Primer_NOMBRE'].strip()
    segundo_nombre=bdi.loc[i,'Segundo_NOMBRE'].strip()
    apellido=bdi.loc[i,'PRIMER APELLIDO'].strip()
    apellido_completo=apellido+' '+bdi.loc[i,'SEGUNDO APELLIDO'].strip()
    #1. Busqueda en completo
    df=search_orcid('family-name:{}+AND+given-names:{}+AND+affiliation-org-name:{}'.format(
                  apellido_completo.replace(' ','+'), 
                  nombre_completo.replace(' ','+'), affiliation.replace(' ','+') ) ,
                search_token)
    if not df.empty:
        df['forma']=1
    
    #2. Búsqueda de primer nombre y primer appellido
    if df.empty:
        df=search_orcid('family-name:{}+AND+given-names:{}+AND+affiliation-org-name:{}'.format(
                  apellido.replace(' ','+'), 
                  nombre.replace(' ','+'), affiliation.replace(' ','+') ) ,
                search_token)
        if not df.empty:
            df['forma']=2

        #3. Búsqueda de segundo nombre y primer appellido     
        if df.empty and segundo_nombre:
            df=search_orcid('family-name:{}+AND+given-names:{}+AND+affiliation-org-name:{}'.format(
                  apellido.replace(' ','+'), 
                  segundo_nombre.replace(' ','+'), affiliation.replace(' ','+') ) ,
                search_token)
            if not df.empty:
                df['forma']=3        

    if i==5:
        break
                
    if df.empty:
        continue 
    else:
        #inentar verificar y si funciona:
        df['NOMBRE COMPLETO']=bdi.loc[i,'NOMBRE COMPLETO'].strip()
        dft=dft.append(df.loc[[0]]).reset_index(drop=True)
        


0
1
2
3
4
5


In [51]:
bdi[:5]

,CÉDULA,NOMBRE COMPLETO,NOMBRES,PRIMER APELLIDO,SEGUNDO APELLIDO,FACULTAD,DEPARTAMENTO,GRUPO,Primer_NOMBRE,Segundo_NOMBRE
0,71621971,Bernardo Maria Bustamante Cardona,Bernardo Maria,Bustamante,Cardona,Facultad de Artes,Departamento de Artes Visuales,Arteducación,Bernardo,Maria
1,71652139,Carlos Mario Jaramillo Ramirez,Carlos Mario,Jaramillo,Ramirez,Facultad de Artes,Departamento de Artes Visuales,DIDARTE. Didáctica de las artes,Carlos,Mario
2,43613390,Isabel Cristina Restrepo Acevedo,Isabel Cristina,Restrepo,Acevedo,Facultad de Artes,Departamento de Artes Visuales,Hipertrópico: Convergencia entre Arte y Tecnol...,Isabel,Cristina
3,43986489,Alexandra Milena Tabares Garcia,Alexandra Milena,Tabares,Garcia,Facultad de Artes,Departamento de Artes Visuales,Hipertrópico: Convergencia entre Arte y Tecnol...,Alexandra,Milena
4,15257718,Bernardo Barragan Castrillon,Bernardo,Barragan,Castrillon,Facultad de Artes,Departamento de Artes Visuales,Historia de la Práctica Pedagógica en Colombia,Bernardo,


In [52]:
dft[:5]

,num-found,result,forma,NOMBRE COMPLETO
0,2,{'orcid-identifier': {'uri': 'https://orcid.or...,3,Bernardo Maria Bustamante Cardona
1,2,{'orcid-identifier': {'uri': 'https://orcid.or...,1,Carlos Mario Jaramillo Ramirez
2,1,{'orcid-identifier': {'uri': 'https://orcid.or...,1,Isabel Cristina Restrepo Acevedo


Estraer el link de orcid:

Obtener la colunna results:

In [81]:
dft["result"]

0    {'orcid-identifier': {'uri': 'https://orcid.or...
1    {'orcid-identifier': {'uri': 'https://orcid.or...
2    {'orcid-identifier': {'uri': 'https://orcid.or...
Name: result, dtype: object

Notar que es una lista de diccionarios. Por lo tanto se puede convertir a DataFrame. 

Primero extraemos un arreglo que es una lista generalizada con el método `.values`

In [88]:
dft["result"].values 

array([{'orcid-identifier': {'uri': 'https://orcid.org/0000-0002-3546-2163', 'path': '0000-0002-3546-2163', 'host': 'orcid.org'}},
       {'orcid-identifier': {'uri': 'https://orcid.org/0000-0003-4969-8457', 'path': '0000-0003-4969-8457', 'host': 'orcid.org'}},
       {'orcid-identifier': {'uri': 'https://orcid.org/0000-0002-8235-3525', 'path': '0000-0002-8235-3525', 'host': 'orcid.org'}}],
      dtype=object)

Debemos convertir el arreglo en una lista de Python con la funciòn `list()`

In [89]:
list(  dft["result"].values )

[{'orcid-identifier': {'host': 'orcid.org',
   'path': '0000-0002-3546-2163',
   'uri': 'https://orcid.org/0000-0002-3546-2163'}},
 {'orcid-identifier': {'host': 'orcid.org',
   'path': '0000-0003-4969-8457',
   'uri': 'https://orcid.org/0000-0003-4969-8457'}},
 {'orcid-identifier': {'host': 'orcid.org',
   'path': '0000-0002-8235-3525',
   'uri': 'https://orcid.org/0000-0002-8235-3525'}}]

Ahora Pandas lo puede interpretar como un DataFrame

In [95]:
dfresults= pd.DataFrame(  list( dft["result"].values ) ) 
dfresults[:3]

,orcid-identifier
0,"{'uri': 'https://orcid.org/0000-0002-3546-2163', 'path': '0000-0002-3546-2163', 'host': 'orcid.org'}"
1,"{'uri': 'https://orcid.org/0000-0003-4969-8457', 'path': '0000-0003-4969-8457', 'host': 'orcid.org'}"
2,"{'uri': 'https://orcid.org/0000-0002-8235-3525', 'path': '0000-0002-8235-3525', 'host': 'orcid.org'}"


Repetimos el proceso para la columna `"orcid-identifier"`

In [96]:
dfresults=pd.DataFrame( list(  dfresults["orcid-identifier"].values ) )
dfresults[:3ors[0]]

,host,path,uri
0,orcid.org,0000-0002-3546-2163,https://orcid.org/0000-0002-3546-2163
1,orcid.org,0000-0003-4969-8457,https://orcid.org/0000-0003-4969-8457
2,orcid.org,0000-0002-8235-3525,https://orcid.org/0000-0002-8235-3525


In [97]:
dfresults['uri']

0    https://orcid.org/0000-0002-3546-2163
1    https://orcid.org/0000-0003-4969-8457
2    https://orcid.org/0000-0002-8235-3525
Name: uri, dtype: object

Como es una columna de cadena de caràcteres de caracteres, podemos usar el método `.str.split`, para convertirlo en lista con respecto a un separador

In [100]:
dfresults['uri'].str.split('/')

0    [https:, , orcid.org, 0000-0002-3546-2163]
1    [https:, , orcid.org, 0000-0003-4969-8457]
2    [https:, , orcid.org, 0000-0002-8235-3525]
Name: uri, dtype: object

Para extraer una columna i, se usa el mètrodo `.str` con el llamado a la columna i. Para i=3. Como en este caso es el último elemento, también podemos usar -1

In [0]:
ors=dfresults['uri'].str.split('/').str[-1]

# Temporal
Ensayo para `ors[0]`

In [105]:
i=0
ors[i]

'0000-0002-3546-2163'

orcid

In [109]:
kk=search_orcid('orcid:{}'.format(ors[i]),search_token)
kk

,num-found,result
0,1,"{'orcid-identifier': {'uri': 'https://orcid.org/0000-0002-3546-2163', 'path': '0000-0002-3546-2163', 'host': 'orcid.org'}}"


http://members.orcid.org/api/tutorial/read-orcid-records
https://github.com/ORCID/ORCID-Source/blob/master/orcid-model/src/main/resources/record_2.0/README.md#read-sections

## Appendix

In [0]:
r=requests.get('http://docs.python-requests.org/en/master/')

In [0]:
print( r.text[:193] )


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
  "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
  <head>
    
